In [1]:
import numpy as np 
import SlideRunner.general.dependencies
from SlideRunner.dataAccess.database import Database
from SlideRunner.dataAccess.annotations import *
import os
import openslide
import sqlite3
import cv2
import sys
from pathlib import Path
import pandas as pd

In [2]:
DB = Database()

vp = ViewingProfile()
vp.majorityClassVote=True

cm=np.zeros((7,7))
if len(sys.argv)<2:
    print('Synopsis: ',sys.argv[0],'<valrun>')
    sys.exit()
threshold = 5

disagreedclass = 0
agreedclass = 0
#basepath='../WSI/'
basepath='/Volumes/DA_HDD/CAPSTONE/WSI/'
patchSize=128

In [5]:
Databases = ['MITOS_WSI_CMC_COADEL_TR.sqlite',                    # [0]
            'MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite',                 # [1]    
            'MITOS_WSI_CMC_MEL.sqlite',                           # [2]
            'MITOS_WSI_CMC_ODAEL_TR.sqlite',                      # [3]
            'TUPAC.sqlite',                                       # [4]
            'TUPAC_alternativeLabels_augmented_training.sqlite',  # [5]
            'TUPAC_stitched.sqlite']                              # [6]

Path_DB = Path('../databases/')
Path_WSI = Path('/Volumes/DA_HDD/CAPSTONE/WSI/')

In [8]:
#os.system('mkdir -p Data_CMC%s' % sys.argv[1])
os.system('mkdir -p Data_%s' % Databases[2][:-7])

0

In [9]:
Path_Data = Path('/Data_'+Databases[2][:-7]+'/')

In [10]:
Path_Data

PosixPath('/Data_MITOS_WSI_CMC_MEL')

In [14]:
CON_MEL = sqlite3.connect(str(Path_DB/Databases[2]))
MEL_Classes = pd.read_sql_query('SELECT * FROM Classes', con=CON_MEL)           # Create MEL classes dataframe
print(pd.DataFrame(MEL_Classes))

CON_COADEL = sqlite3.connect(str(Path_DB/Databases[1]))
COADEL_Classes = pd.read_sql_query('SELECT * FROM Classes', con=CON_COADEL)     # Create COADEL classes dataframe
#print(pd.DataFrame(COADEL_Classes))

CON_ODAEL = sqlite3.connect(str(Path_DB/Databases[3]))
ODAEL_Classes = pd.read_sql_query('SELECT * FROM Classes', con=CON_ODAEL)       # Create ODAEL classes dataframe
#print(pd.DataFrame(ODAEL_Classes))


CON_TUPAC = sqlite3.connect(str(Path_DB/Databases[5]))
TUPAC_Classes = pd.read_sql_query('SELECT * FROM Classes', con=CON_TUPAC)       # Create TUPAC classes dataframe
#print(pd.DataFrame(TUPAC_Classes))

   uid                     name    color
0    1  Mitotic cell look-alike  #21ff22
1    2           Mitotic figure  #0000ef


In [13]:
dirs = ['Mitosis', 'Nonmitosis']
for k in dirs:
    os.system('mkdir -p Data_%s/train/%s' % (Databases[2][:-7],k))   # Create a train folder under Mitosis and Nonmitosis folders
    os.system('mkdir -p Data_%s/test/%s' % (Databases[2][:-7],k))    # Create a test folder under Mitosis and Nonmitosis folders

def listOfSlides(DB):
    DB.execute('SELECT uid,filename from Slides')
    return DB.fetchall()

slidelist_test_1 = ['18','3', '22','10','15','21','14']
slidelist_test_2 = ['1', '20','17','5', '2', '11','16']
slidelist_test_3 = ['12','13','7', '19','8', '6', '9']

test_slides = { '1': slidelist_test_1,
                '2': slidelist_test_2,
                '3': slidelist_test_3}

In [19]:
MEL_Anno= pd.read_sql_query('SELECT * FROM Annotations', con=CON_MEL)    # Includes slide # (uid), agreedClass (annotation, i.e. mitotic/nonmitotic)

MEL_Slides= pd.read_sql_query('SELECT * FROM Slides', con=CON_MEL)       # Includes slide #, Whole Slide Image file name
MEL_Slides_new = MEL_Slides.rename(columns={'uid':'slide'})              # Change the uid (slide#) with WSI file name              

MEL_merge = pd.merge(MEL_Anno,MEL_Slides_new, on='slide')                # Merge MEL_Slides with MEL_Anno - this is to see which slide # is associated with WSI file name
class1 = pd.DataFrame(MEL_merge[['slide','filename']][MEL_merge['agreedClass']==1].value_counts().sort_index())     # Pull together sum of agreedClass == 1 (nonmitotic) for each slide
class1 = class1.reset_index().rename(columns={0:'Mitotic cell look-alike'})                                         # Rename count column with agreedClass name
class2 = pd.DataFrame(MEL_merge[['slide','filename']][MEL_merge['agreedClass']==2].value_counts().sort_index())     # Pull together sum of agreedClass == 2 (mitotic) for each slide
class2 = class2.reset_index().rename(columns={0:'Mitotic figure'})                                                  # Rename count column with agreedClass name

MEL_class_total = pd.merge(class1, class2)
MEL_class_total                                                         # list of Whole Slide Images's slide # and file name along with the number of nonmitotic and mitotic counts

,slide,filename,Mitotic cell look-alike,Mitotic figure
0,1,a8773be388e12df89edd.svs,204,64
1,2,460906c0b1fe17ea5354.svs,1199,673
2,3,2d56d1902ca533a5b509.svs,228,139
3,5,d0423ef9a648bb66a763.svs,1143,378
4,6,50cf88e9a33df0c0c8f9.svs,1632,1197
5,7,da18e7b9846e9d38034c.svs,991,187
6,8,d7a8af121d7d4f3fbf01.svs,1832,716
7,9,2191a7aa287ce1d5dbc0.svs,2155,1449
8,10,69a02453620ade0edefd.svs,1407,634
9,11,c4b95da36e32993289cb.svs,2454,1097


In [29]:
print('Total count of mitotic figures is ' + str(MEL_class_total['Mitotic figure'].sum()))
print('Total count of nonmitotic figures is ' + str(MEL_class_total['Mitotic cell look-alike'].sum()))
print('Sum of all annotations is ' + str(MEL_class_total['Mitotic figure'].sum() + MEL_class_total['Mitotic cell look-alike'].sum()))

Total count of mitotic figures is 13342
Total count of nonmitotic figures is 26526
Sum of all annotations is 39868


In [30]:
info_data = []    # This is to collect tile image ID, its associated WSI file name, mitotic figure radius, whether the tile is in test or train folder
from os import system
DB.open('../databases/'+Databases[2])    # Make sure to select the appropriate database, check above list to see the names

for slideid,filename in listOfSlides(DB):
    DB.loadIntoMemory(slideid)
    
    
    slide=openslide.open_slide(basepath+filename)

    for k in DB.annotations.keys():

        anno = DB.annotations[k]

        if anno.deleted or anno.annotationType != AnnotationType.SPOT:
            continue
        coord_x = anno.x1     # x coordinate of the annotation
        coord_y = anno.y1     # y coordinate of the annotation
        radius = anno.r       # radius of the annotation 

        lu_x = int(coord_x - int(patchSize/2))     # this indicates the left upper x coordinate
        lu_y = int(coord_y - int(patchSize/2))     # this indicates the left upper y coordinate
        img = np.array(slide.read_region(location=(lu_x, lu_y), level=0, size=(patchSize, patchSize)))
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)



        #istest = 'train/' if str(slideid) not in test_slides[sys.argv[1]] else 'test/'
        istest = 'train/' if str(slideid) not in test_slides else 'test/'

        info_data.append([k, slideid, filename, anno.agreedClass, radius, istest])

        if (anno.agreedClass ==2):
            temp1 = (str('.') + str(Path_Data) + '/' + istest + 'Mitosis/%d.png' % k)
            cv2.imwrite(temp1, img)
            

        if (anno.agreedClass==1):
            temp2 = (str('.') + str(Path_Data) + '/' + istest + 'Nonmitosis/%d.png' % k)
            cv2.imwrite(temp2, img)

In [31]:
len(pd.DataFrame(info_data)[2].unique())

21

In [36]:
info_data_pd = pd.DataFrame(info_data).rename(columns={0:'Tile_num', 1:'SlideID', 2:'Slide_file_name', 3:'agreedClass', 4:'anno_radius', 5:'test_train_folder' }) 

In [45]:
# info_data_pd['SlideID'].value_counts().sort_index()

In [44]:
#MEL_class_total['SUM'] = MEL_class_total['Mitotic cell look-alike'] + MEL_class_total['Mitotic figure']
#MEL_class_total

In [46]:
test_slides

{'1': ['18', '3', '22', '10', '15', '21', '14'],
 '2': ['1', '20', '17', '5', '2', '11', '16'],
 '3': ['12', '13', '7', '19', '8', '6', '9']}

In [55]:
info_data_pd[['SlideID','Slide_file_name','test_train_folder']].value_counts().sort_index()

SlideID  Slide_file_name           test_train_folder
1        a8773be388e12df89edd.svs  test/                 268
2        460906c0b1fe17ea5354.svs  test/                1872
3        2d56d1902ca533a5b509.svs  test/                 367
5        d0423ef9a648bb66a763.svs  train/               1521
6        50cf88e9a33df0c0c8f9.svs  train/               2829
7        da18e7b9846e9d38034c.svs  train/               1178
8        d7a8af121d7d4f3fbf01.svs  train/               2548
9        2191a7aa287ce1d5dbc0.svs  train/               3604
10       69a02453620ade0edefd.svs  train/               2041
11       c4b95da36e32993289cb.svs  train/               3551
12       fa4959e484beec77543b.svs  train/               5547
13       72c93e042d0171a61012.svs  train/                505
14       4eee7b944ad5e46c60ce.svs  train/                161
15       b1bdee8e5e3372174619.svs  train/               2072
16       3d3d04eca056556b0b26.svs  train/               3493
17       084383c18b9060880e82.sv

In [58]:
info_data_pd['test_train_folder'].value_counts()

train/    37360
test/      2507
Name: test_train_folder, dtype: int64

In [66]:
train_tiles_count = len(os.listdir('./Data_MITOS_WSI_CMC_MEL/train/Mitosis/')) + len(os.listdir('./Data_MITOS_WSI_CMC_MEL/train/Nonmitosis/'))
test_tiles_count = len(os.listdir('./Data_MITOS_WSI_CMC_MEL/test/Mitosis/')) + len(os.listdir('./Data_MITOS_WSI_CMC_MEL/test/Nonmitosis/'))

In [70]:
print('train_tiles_count = ' + str(train_tiles_count))
print('test_tiles_count = ' + str(test_tiles_count))

train_tiles_count = 37360
test_tiles_count = 2507
